In [25]:
import os

os.environ["PYICEBERG_HOME"] = os.getcwd()

In [26]:
from pyiceberg.catalog import load_catalog

In [27]:
from pathlib import Path
warehouse_path = Path("iceberg_catalog").absolute()
catalog = load_catalog(
    "local",
    **{
        "type": "sql",
        "uri": "sqlite:///iceberg_catalog/catalog.db",
        "warehouse": f"file://{warehouse_path}"
    }
)

In [28]:
print(catalog.properties)

{'uri': 'sqlite:///iceberg_catalog/catalog.db', 'warehouse': 'file:///home/jupyter-jj/gates-hands-on-training/iceberg-installation/iceberg_catalog', 'type': 'sql'}


# Creating and Selecting Records

In [29]:
from pyiceberg .schema import Schema
from pyiceberg.types import NestedField, IntegerType, StringType, BooleanType, DoubleType
from pyiceberg.partitioning import PartitionSpec
import pandas

In [30]:
catalog.create_namespace_if_not_exists('Sales')

In [31]:
product_schema = Schema(
    NestedField(field_id=1, name='ProductId', field_type=IntegerType(), required=True),
    NestedField(field_id=2, name='Name', field_type=StringType(), required=True),
    NestedField(field_id=3, name='Category', field_type=StringType(), required=True),
    NestedField(field_id=4, name='Price', field_type=DoubleType(), required=True),
    NestedField(field_id=5, name='Stock', field_type=IntegerType(), required=True),
    NestedField(field_id=6, name='IsActive', field_type=BooleanType(), required=True)
)

In [32]:
product_table = catalog.create_table_if_not_exists(
    identifier="Sales.Product",
    schema=product_schema
)

In [33]:
import pyarrow as pa


product_data = pa.Table.from_pylist([
    {"ProductId": 1, "Name": "Laptop", "Category": "Electronics", "Price": 999.99, "Stock": 15, "IsActive": True},
    {"ProductId": 2, "Name": "Smartphone", "Category": "Electronics", "Price": 699.99, "Stock": 30, "IsActive": True},
    {"ProductId": 3, "Name": "Tablet", "Category": "Electronics", "Price": 399.99, "Stock": 20, "IsActive": True},
    {"ProductId": 4, "Name": "Smartwatch", "Category": "Wearable", "Price": 199.99, "Stock": 25, "IsActive": True},
    {"ProductId": 5, "Name": "Wireless Earbuds", "Category": "Accessories", "Price": 129.99, "Stock": 50, "IsActive": True},
    {"ProductId": 6, "Name": "Bluetooth Speaker", "Category": "Accessories", "Price": 89.99, "Stock": 35, "IsActive": True},
    {"ProductId": 7, "Name": "Gaming Console", "Category": "Gaming", "Price": 499.99, "Stock": 0, "IsActive": False},
    {"ProductId": 8, "Name": "Mechanical Keyboard", "Category": "Accessories", "Price": 79.99, "Stock": 40, "IsActive": True},
    {"ProductId": 9, "Name": "Gaming Mouse", "Category": "Accessories", "Price": 59.99, "Stock": 45, "IsActive": True},
    {"ProductId": 10, "Name": "Monitor", "Category": "Electronics", "Price": 249.99, "Stock": 20, "IsActive": True},
    {"ProductId": 11, "Name": "External Hard Drive", "Category": "Storage", "Price": 119.99, "Stock": 30, "IsActive": True},
    {"ProductId": 12, "Name": "USB Flash Drive", "Category": "Storage", "Price": 19.99, "Stock": 0, "IsActive": False},
    {"ProductId": 13, "Name": "Power Bank", "Category": "Accessories", "Price": 39.99, "Stock": 60, "IsActive": True},
    {"ProductId": 14, "Name": "VR Headset", "Category": "Gaming", "Price": 299.99, "Stock": 8, "IsActive": True},
    {"ProductId": 15, "Name": "Smart TV", "Category": "Electronics", "Price": 799.99, "Stock": 12, "IsActive": True},
    {"ProductId": 16, "Name": "Wireless Router", "Category": "Networking", "Price": 129.99, "Stock": 22, "IsActive": True},
    {"ProductId": 17, "Name": "Digital Camera", "Category": "Photography", "Price": 549.99, "Stock": 18, "IsActive": True},
    {"ProductId": 18, "Name": "Tripod Stand", "Category": "Photography", "Price": 49.99, "Stock": 0, "IsActive": False},
    {"ProductId": 19, "Name": "Fitness Tracker", "Category": "Wearable", "Price": 89.99, "Stock": 27, "IsActive": True},
    {"ProductId": 20, "Name": "Electric Toothbrush", "Category": "Health", "Price": 59.99, "Stock": 40, "IsActive": True},
    {"ProductId": 21, "Name": "Hair Dryer", "Category": "Personal Care", "Price": 39.99, "Stock": 28, "IsActive": True},
    {"ProductId": 22, "Name": "Air Purifier", "Category": "Home Appliances", "Price": 179.99, "Stock": 0, "IsActive": False},
    {"ProductId": 23, "Name": "Microwave Oven", "Category": "Home Appliances", "Price": 249.99, "Stock": 14, "IsActive": True},
    {"ProductId": 24, "Name": "Washing Machine", "Category": "Home Appliances", "Price": 499.99, "Stock": 7, "IsActive": True},
    {"ProductId": 25, "Name": "Refrigerator", "Category": "Home Appliances", "Price": 999.99, "Stock": 0, "IsActive": False}
], schema=product_table.schema().as_arrow())

In [34]:
product_table.append(df=product_data)

In [35]:
product_table.scan().select('ProductId', 'Price').to_arrow().to_pandas()

,ProductId,Price
0,1,999.99
1,2,699.99
2,3,399.99
3,4,199.99
4,5,129.99
5,6,89.99
6,7,499.99
7,8,79.99
8,9,59.99
9,10,249.99


# Selecting Columns

In [36]:
product = catalog.load_table('Sales.Product')

In [37]:
product.scan().select('Name').to_arrow().to_pandas()

,Name
0,Laptop
1,Smartphone
2,Tablet
3,Smartwatch
4,Wireless Earbuds
5,Bluetooth Speaker
6,Gaming Console
7,Mechanical Keyboard
8,Gaming Mouse
9,Monitor


# Filter

In [38]:
product = catalog.load_table('Sales.Product')

In [39]:
from pyiceberg.expressions import EqualTo, GreaterThan, In, NotIn

In [40]:
product.scan(row_filter= GreaterThan('ProductId', 10)).to_arrow().to_pandas()

,ProductId,Name,Category,Price,Stock,IsActive
0,11,External Hard Drive,Storage,119.99,30,True
1,12,USB Flash Drive,Storage,19.99,0,False
2,13,Power Bank,Accessories,39.99,60,True
3,14,VR Headset,Gaming,299.99,8,True
4,15,Smart TV,Electronics,799.99,12,True
5,16,Wireless Router,Networking,129.99,22,True
6,17,Digital Camera,Photography,549.99,18,True
7,18,Tripod Stand,Photography,49.99,0,False
8,19,Fitness Tracker,Wearable,89.99,27,True
9,20,Electric Toothbrush,Health,59.99,40,True


In [41]:
product.scan(row_filter= EqualTo('Name', 'Smart TV')).to_arrow().to_pandas()

,ProductId,Name,Category,Price,Stock,IsActive
0,15,Smart TV,Electronics,799.99,12,True


In [42]:
product.scan(row_filter= NotIn('ProductId', [2,3,4])).to_arrow().to_pandas()

,ProductId,Name,Category,Price,Stock,IsActive
0,1,Laptop,Electronics,999.99,15,True
1,5,Wireless Earbuds,Accessories,129.99,50,True
2,6,Bluetooth Speaker,Accessories,89.99,35,True
3,7,Gaming Console,Gaming,499.99,0,False
4,8,Mechanical Keyboard,Accessories,79.99,40,True
5,9,Gaming Mouse,Accessories,59.99,45,True
6,10,Monitor,Electronics,249.99,20,True
7,11,External Hard Drive,Storage,119.99,30,True
8,12,USB Flash Drive,Storage,19.99,0,False
9,13,Power Bank,Accessories,39.99,60,True
